## TL;DR: End-to-End Usage Example
```
from kaggle.competitions import nflrush
env = nflrush.make_env()

# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
train_my_model(train_df)

for (test_df, sample_prediction_df) in env.iter_test():
  predictions_df = make_my_predictions(test_df, sample_prediction_df)
  env.predict(predictions_df)
  
env.write_submission_file()
```
Note that `train_my_model` and `make_my_predictions` are functions you need to write for the above example to work.

In [ ]:
from kaggle.competitions import nflrush
import pandas as pd

# You can only call make_env() once, so don't lose it!
env = nflrush.make_env()

In [ ]:
df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
df.head()

In [ ]:
df.OffenseFormation.fillna(df.OffenseFormation.value_counts().idxmax(), inplace=True)

In [ ]:
df.DefendersInTheBox.fillna(df.DefendersInTheBox.value_counts().idxmax(), inplace=True)

In [ ]:
def ftinch2cm(height_ftinch):
    h_ft = int(height_ftinch[0])
    h_inch = int(height_ftinch[-1])
    return round(((h_ft * 12) + h_inch) * 2.54, 1)

df.PlayerHeight = df.PlayerHeight.apply(ftinch2cm)

In [ ]:
df['IsRusher'] = df['NflId'] == df['NflIdRusher']
df_NflIdRusher = df[df.IsRusher == True]

In [ ]:
df_NflIdRusher.reset_index(drop=True, inplace=True)

In [ ]:
cols_coba = ['Dis', 'OffenseFormation', 'DefendersInTheBox', 'PlayerHeight', 'PlayerWeight']
df_new = df_NflIdRusher[cols_coba]

In [ ]:
yards = df_NflIdRusher.Yards

In [ ]:
df_fix = pd.concat([df_new.drop(['OffenseFormation'], axis=1), pd.get_dummies(df_new['OffenseFormation'], prefix='Formation')], axis=1)

In [ ]:
import numpy as np

X = np.asarray(df_fix)
y = np.asarray(yards)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=45,
                                                    test_size=.25)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=250, random_state=45,
                            max_depth=10)
rfr.fit(X_train, y_train)

In [ ]:
rfr.score(X_test, y_test)

sampai sini adalah contoh persiapan data

In [ ]:
# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.

# iter_test = env.iter_test()

In [ ]:
# (test_df, sample_prediction_df) = next(iter_test)
# test_df

In [ ]:
# sample_prediction_df

In [ ]:
# a = test_df
# a['IsRusher'] = a['NflId'] == a['NflIdRusher']
# a = a[a.IsRusher == True]
# a = a[cols_coba]
# a.PlayerHeight = a.PlayerHeight.apply(ftinch2cm)
# a = pd.concat([a.drop(['OffenseFormation'], axis=1), pd.get_dummies(a['OffenseFormation'], prefix='Formation')], axis=1)
# missing_cols = set( df_fix.columns ) - set( a.columns )
# for c in missing_cols:
#     a[c] = 0
# a = a[df_fix.columns]
# X = np.asarray(a)
# y = round(rfr.predict(X)[0])
# op = [0 if x - y < 0 else 1 for x in np.arange(-99, 100)]
# yardsgain = pd.DataFrame(data=[op],columns=sample_prediction_df.columns)
# yardsgain[yardsgain.columns[98:108]]

def make_prediction(df, sample, env, model):
    
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    df = df[df.IsRusher == True]
    df = df[['Dis', 'OffenseFormation', 'DefendersInTheBox', 'PlayerHeight', 'PlayerWeight']]
    df.PlayerHeight = df.PlayerHeight.apply(ftinch2cm)
    df = pd.concat([df.drop(['OffenseFormation'], axis=1), pd.get_dummies(df['OffenseFormation'], prefix='Formation')], axis=1)
    missing_cols = set(df_fix.columns) - set(df.columns)
    for c in missing_cols:
        df[c] = 0
    df = df[df_fix.columns]
    X = np.asarray(df)
    y = round(model.predict(X)[0])
    y_pred = [0 if x - y < 0 else 1 for x in np.arange(-99, 100)]
    env.predict(pd.DataFrame(data=[y_pred], columns=sample.columns))
    return np.asarray(y_pred)

In [ ]:
from tqdm import tqdm

In [ ]:
for test, sample in tqdm(env.iter_test()):
    make_prediction(test, sample, env, rfr)

In [ ]:
env.write_submission_file()